In [ ]:
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.imagenet_utils import decode_predictions
from tensorflow.keras.applications import resnet50
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf

In [ ]:
batch_size = 9
img_height = 224
img_width = 224
epochs = 10

In [ ]:
data_dir = "../../data/train"

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=119,
    image_size=(img_height, img_width),
    batch_size=batch_size)


val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

In [ ]:
img_shape = (img_height , img_width, 3)

VGG16_MODEL = tf.keras.applications.VGG16(input_shape = img_shape,
                                         include_top = False,
                                         weights = 'imagenet')

VGG16_MODEL.trainable=False

# flatten이 없음 ( globalaveragepooling으로 대체 )
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
class_names = train_ds.class_names

# FFNN의 가중치는 학습됨
prediction_layer = tf.keras.layers.Dense(len(class_names),
                                         activation ='softmax' )

In [ ]:
model = tf.keras.Sequential([
    VGG16_MODEL,
    global_average_layer,
    prediction_layer
])

In [ ]:
model.compile(optimizer = 'adam',
             loss = 'sparse_categorical_crossentropy',
             metrics=['accuracy'])
history = model.fit(train_ds, epochs=epochs,
                   validation_steps=2,validation_data=(val_ds))

In [ ]:
model.save("../../checkpoints/saved_model/custom_model")